In [2]:
import optuna

## 핵심
- Search할 Parameter가 증가할수록 필요한 시간이 exponentially하게 증가한다.
- 중요한 Parameter에 대해서만 Search하도록 한다.

## Pythonic Search Space

In [3]:
# 다음과 같은 방법으로 Search Space를 정하여 탐색을 진행할 수 있다.
def objective(trial):
    # Categorical parameter
    optimizer = trial.suggest_categorical("optimizer", ["MomentumSGD", "Adam"])

    # Integer parameter
    num_layers = trial.suggest_int("num_layers", 1, 3)

    # Integer parameter (log)
    num_channels = trial.suggest_int("num_channels", 32, 512, log=True)

    # Integer parameter (discretized)
    num_units = trial.suggest_int("num_units", 10, 100, step=5)

    # Floating point parameter
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 1.0)

    # Floating point parameter (log)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)

    # Floating point parameter (discretized)
    drop_path_rate = trial.suggest_float("drop_path_rate", 0.0, 1.0, step=0.1)

## Defining Parameter Spaces

In [4]:
# if문을 활용하여 Conditional하게 search space를 지정할 수 있다.
# 이를 branches를 활용한 방법이라 한다.
import sklearn.ensemble
import sklearn.svm

def objective(trial):
    classifier_name = trial.suggest_categorical('classifier', ['SVC', 'RandomForest']) # SVC와 RandomForest 중 택1

    if classifier_name == 'SVC': # SVC일 경우 if문에서의 parameter 중 선택
        svc_c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True) # 1e-10 ~ 1e10 중 선택
        classifier_obj = sklearn.svm.SVC(C=svc_c)
    else:
        rf_max_depth = trial.suggest_int('rf_max_depth', 2, 32, log=True)
        classifier_obj = sklearn.ensemble.RandomForestClassifier(max_deptn=rf_max_depth)

In [ ]:
import torch
import torch.nn as nn

def creat_model(trial, in_size):
    '''
    for loop를 활용해서 각 layer마다 다른 parameter를 활용할 수 있다.
    '''
    n_layers = trial.suggest_int('n_layers', 1, 3)

    layers = []
    for i in range(n_layers) :
        n_units = trial.suggest_int('n_units_{}'.format(i), 4, 128, log=True)
        layers.append(nn.Linear(in_size, n_units))
        layers.append(nn.ReLU())
        in_size = n_units
    layers.append(nn.Linear(in_size, 10))

    return nn.Sequential(*layers)